# Libraries

In [4]:
import numpy as np
import pandas as pd
import os
import time
import csv
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, DepthwiseConv2D
from keras.models import model_from_json

2023-05-12 21:40:07.765890: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-12 21:40:07.821362: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 21:40:09.706812: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Settings

In [3]:
# Normal Data Path
face_age_path = '../02_Data/face_age' # Added path to gitingnore, you will have to add data to this path

# Augmented Data Path
augmented_data_path = '../02_Data/augmented_data' # Added path to gitingnore, you will have to add data to this path

In [2]:
# Training sets
raw_train_path = "../02_Data/face_age_data/augmented_data_train"
aug_train_path = "../02_Data/face_age_data/face_age_balanced_train"


# Validation set
raw_val_path = "../02_Data/face_age_data/face_age_balanced_val"

# Testing set
raw_test_path = "../02_Data/face_age_data/face_age_balanced_test"

# Image Manipulation

In [5]:
def load_data(folders, img_size=(200, 200)):
    X = []
    y = []
    for folder_path in folders:
        for folder in os.listdir(folder_path):
            if os.path.isdir(os.path.join(folder_path, folder)):
                age = int(folder.replace("aug_", ""))
                for file in os.listdir(os.path.join(folder_path, folder)):
                    img_path = os.path.join(folder_path, folder, file)
                    img = Image.open(img_path)
                    img = img.resize(img_size)
                    img = np.array(img)
                    X.append(img)
                    y.append(age)
    return np.array(X), np.array(y)

folder_paths = [raw_train_path, aug_train_path]
img_size = (200, 200)
X_train, y_train = load_data(folder_paths, img_size)

In [ ]:
# Normalize pixel values
X_train = X_train / 255.0

In [15]:
print(
    f"Shape of X: {X_train.shape}", "\n",
    f"- {X_train.shape[0]}: Number of images in the dataset", "\n",
    f"- {X_train.shape[1]}: Height of each image", "\n",
    f"- {X_train.shape[2]}: Width of each image", "\n",
    f"- {X_train.shape[3]}: Number of channels of each image (Red, Green, and Blue)"
    )

Shape of X: (45000, 200, 200, 3) 
 - 45000: Number of images in the dataset 
 - 200: Height of each image 
 - 200: Width of each image 
 - 3: Number of channels of each image (Red, Green, and Blue)


In [13]:
print(f"Labels: {y}")

Labels: [32 32 32 ... 58 58 58]


In [17]:
print(
    f"Size of arrays:", "\n",
    f"- X_train shape: {X_train.shape}", "\n",
    f"- y_train shape: {y_train.shape}", "\n",
    )

NameError: name 'X_train' is not defined

In [8]:
# Save train test split
np.save("../02_Data/augmented_traintest_split/X_train.npy", X_train)
np.save("../02_Data/augmented_traintest_split/X_test.npy", X_test)
np.save("../02_Data/augmented_traintest_split/y_train.npy", y_train)
np.save("../02_Data/augmented_traintest_split/y_test.npy", y_test)

# # Load saved train-test split data
# X_train = np.load("../02_Data/face_age_traintest_split/X_test.npy")
# X_test = np.load("../02_Data/face_age_traintest_split/X_test.npy")
# y_train = np.load("../02_Data/face_age_traintest_split/y_train.npy")
# y_test = np.load("../02_Data/face_age_traintest_split/y_test.npy")

# Model Setup

#### Check for GPUs

In [9]:
import tensorflow as tf

# Check if GPUs are available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  5


#### Set up training

In [10]:
# Training parameters
epochs = 15
batch_size = 32
optimizer = "adam"
loss = "mean_squared_error"
metrics = ['mae']

In [11]:
import tensorflow as tf
from keras.layers import Conv2D, LeakyReLU, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, Add
from keras.models import Model

# Create a MirroredStrategy
strategy = tf.distribute.MirroredStrategy(devices=["GPU:0", "GPU:1", "GPU:2", "GPU:3", "GPU:4"])

# Residual block
def residual_block(input_layer, filters):
    x = Conv2D(filters, (3, 3), padding='same')(input_layer)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Add()([x, input_layer])
    return x

def create_model(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)

    x = Conv2D(32, (3, 3), padding='same')(input_layer)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    for _ in range(3):
        x = residual_block(x, 32)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)

    for _ in range(4):
        x = residual_block(x, 64)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)

    for _ in range(6):
        x = residual_block(x, 128)
    x = MaxPooling2D((2, 2))(x)
    
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)

    for _ in range(8):
        x = residual_block(x, 256)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(2048)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    output = Dense(1)(x)

    model = Model(inputs=input_layer, outputs=output)

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

# Open the strategy scope
with strategy.scope():
    input_shape = (img_size[0], img_size[1], 3)
    model = create_model(input_shape)

2023-05-11 19:08:24.698664: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46671 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:1a:00.0, compute capability: 8.6
2023-05-11 19:08:24.699521: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46671 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:1b:00.0, compute capability: 8.6
2023-05-11 19:08:24.700612: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 46671 MB memory:  -> device: 2, name: NVIDIA RTX A6000, pci bus id: 0000:88:00.0, compute capability: 8.6
2023-05-11 19:08:24.701242: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 46671 MB memory:  -> device: 3, name: NVIDIA RTX A6000, pci bus id: 0000:89:00.0, 

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:te

#### Train model

In [12]:
# Train the model with your data
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1)

2023-05-11 19:10:11.278190: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [40169]
	 [[{{node Placeholder/_11}}]]
2023-05-11 19:10:11.278538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [40169,200,200,3]
	 [[{{node Placeholder/_10}}]]


Epoch 1/15
INFO:tensorflow:batch_all_reduce: 202 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 202 all-reduces with algorithm = nccl, num_packs = 1


2023-05-11 19:11:38.994501: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-11 19:11:41.777751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-11 19:11:44.606841: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-11 19:11:47.365281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-11 19:11:49.439078: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-11 19:11:51.117849: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-11 19:11:54.632056: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f1200b9e9d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-11 19:11:54.632098: I tens

1256/1256 [==============================] - ETA: 0s - loss: 1084.2892 - mae: 24.9366

2023-05-11 19:16:26.826581: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [10043]
	 [[{{node Placeholder/_11}}]]
2023-05-11 19:16:26.827005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [10043]
	 [[{{node Placeholder/_11}}]]


1256/1256 [==============================] - 396s 213ms/step - loss: 1084.2892 - mae: 24.9366 - val_loss: 559.2368 - val_mae: 18.4882
Epoch 2/15
1256/1256 [==============================] - 239s 191ms/step - loss: 358.0513 - mae: 15.0227 - val_loss: 558.6927 - val_mae: 20.0817
Epoch 3/15
1256/1256 [==============================] - 239s 190ms/step - loss: 319.9090 - mae: 14.1685 - val_loss: 206.0069 - val_mae: 10.8920
Epoch 4/15
1256/1256 [==============================] - 240s 191ms/step - loss: 299.5709 - mae: 13.6775 - val_loss: 136.7419 - val_mae: 9.2706
Epoch 5/15
1256/1256 [==============================] - 240s 191ms/step - loss: 275.7387 - mae: 13.1473 - val_loss: 298.8853 - val_mae: 14.1575
Epoch 6/15
1256/1256 [==============================] - 238s 189ms/step - loss: 270.7150 - mae: 13.0031 - val_loss: 289.4883 - val_mae: 13.5581
Epoch 7/15
1256/1256 [==============================] - 239s 190ms/step - loss: 275.0542 - mae: 13.0913 - val_loss: 274.2598 - val_mae: 13.4239
Epo

# Save, Load, and Display Model History

In [14]:
def append_model_history_to_file(file_name, description, input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae):
    train_mae = round(train_mae, 6)
    validation_mae = round(validation_mae, 6)

    if not os.path.exists(file_name):
        with open(file_name, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Model', 'Input Shape', 'Epochs', 'Batch Size', 'Optimizer', 'Loss Function', 'Train MAE', 'Validation MAE'])

    with open(file_name, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        try:
            next(csv_reader)  # Skip the header row
        except StopIteration:
            pass  # The CSV file is empty or only contains the header row

        for row in csv_reader:
            if list(map(str, row[1:])) == list(map(str, [input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae])):
                print("Entry with the same parameters already exists.")
                return False

    with open(file_name, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([description, input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae])

    return True

timestamp = int(time.time())
file_name = '../04_Age_Prediction/model_history.csv'
description = f"{timestamp}"
input_shape = str(X_train.shape[1:])
optimizer = optimizer
loss_function = loss
train_mae = history.history['mae'][-1]
validation_mae = history.history['val_mae'][-1]

new_entry_added = append_model_history_to_file(file_name, description, input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae)

# Save model
def save_model_architecture(model, file_name):
    model_json = model.to_json()
    with open(file_name, "w") as json_file:
        json_file.write(model_json)

if new_entry_added:
    model_architecture_file = f"../04_Age_Prediction/models/{description}.json"
    save_model_architecture(model, model_architecture_file)
else:
    print("Model not saved as an entry with the same parameters already exists.")


Entry with the same parameters already exists.
Model not saved as an entry with the same parameters already exists.


In [15]:
def load_data_to_dataframe(file_name):
    if not os.path.exists(file_name):
        print("File does not exist.")
        return None
    
    df = pd.read_csv(file_name)
    return df

file_name = '../04_Age_Prediction/model_history.csv'
df = load_data_to_dataframe(file_name)

df

,Model,Input Shape,Epochs,Batch Size,Optimizer,Loss Function,Train MAE,Validation MAE
0,1683270923,"(200, 200, 3)",10,16,adam,mean_squared_error,10.241365,8.678753
1,1683286770,"(200, 200, 3)",1000,16,adam,mean_squared_error,6.187893,5.758430
2,1683433197,"(200, 200, 3)",50,32,adam,mean_squared_error,9.656551,7.317583
3,1683438058,"(200, 200, 3)",50,32,adam,mean_squared_error,10.114231,7.153706
4,1683439878,"(200, 200, 3)",50,32,adam,mean_squared_error,8.796492,7.445862
5,1683441408,"(200, 200, 3)",50,32,adam,mean_squared_error,9.388038,6.963918
6,1683442226,"(200, 200, 3)",50,32,adam,mean_squared_error,11.328677,8.919698
7,1683445490,"(200, 200, 3)",50,32,adam,mean_squared_error,9.128469,6.906603
8,1683477636,"(200, 200, 3)",650,32,adam,mean_squared_error,8.564434,19.438639
9,1683835954,"(200, 200, 3)",15,32,adam,mean_squared_error,11.569266,7.070375


In [ ]:
# Specify model to load
model_to_load = "1683270923"

def load_and_display_model_architecture(file_name):
    if not os.path.exists(file_name):
        print("File does not exist.")
        return None

    with open(file_name, "r") as json_file:
        model_json = json_file.read()
    
    model = model_from_json(model_json)
    model.summary()

    return model

model_architecture_file = f"../04_Age_Prediction/models/{model_to_load}.json"
model = load_and_display_model_architecture(model_architecture_file)

# 